Download Data

In [ ]:
# !pip install gdown

#train
!gdown --id 1aLxrqOFzCxp-saWbh-sNxGVSXptj82VS

#val
!gdown --id 1oObOVTTGj7qkBA369WCgr3VaXbvfQpnG

#test
!gdown --id 1U2RWM3qh7woDsOLVYMTUoFwDIM-n1s6I


/usr/local/lib/python3.11/dist-packages/gdown/__main__.py:140: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From: https://drive.google.com/uc?id=1aLxrqOFzCxp-saWbh-sNxGVSXptj82VS
To: /content/train.json
100% 11.1M/11.1M [00:00<00:00, 27.4MB/s]
/usr/local/lib/python3.11/dist-packages/gdown/__main__.py:140: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From: https://drive.google.com/uc?id=1oObOVTTGj7qkBA369WCgr3VaXbvfQpnG
To: /content/valid.json
100% 4.81M/4.81M [00:00<00:00, 28.5MB/s]
/usr/local/lib/python3.11/dist-packages/gdown/__main__.py:140: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From: https://drive.

In [ ]:
!pip install datasets
!pip install bert-score sacrebleu


In [ ]:
import json
import pandas as pd
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
from transformers import Seq2SeqTrainer, Seq2SeqTrainingArguments, DataCollatorForSeq2Seq, TrainerCallback
from peft import get_peft_model, LoraConfig, TaskType
from datasets import Dataset
import torch
import os
from bert_score import score as bert_score
from sacrebleu import corpus_bleu
from collections import defaultdict

### Models T5

In [ ]:
# https://drive.google.com/file/d/16bv1opM4wD-tsgCoRlItTB2_Ffjh3Qr9/view?usp=sharing

In [ ]:
!gdown --id 1Q5TPOUik4e4i6QxjpXXGJx1I75v0En0g

!gdown --id 1fKpXrNflzGcNmOmwHpRxQp3XQdH5W7wF

!gdown --id 1rDZUGRs3SxZ4n5-soPaH4qcJw7uaH1xm

!gdown --id 16bv1opM4wD-tsgCoRlItTB2_Ffjh3Qr9

/usr/local/lib/python3.11/dist-packages/gdown/__main__.py:140: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From (original): https://drive.google.com/uc?id=1Q5TPOUik4e4i6QxjpXXGJx1I75v0En0g
From (redirected): https://drive.google.com/uc?id=1Q5TPOUik4e4i6QxjpXXGJx1I75v0En0g&confirm=t&uuid=0d59cd9f-e9cb-4044-9b60-e43440bccc64
To: /content/sug_model_moe_t5.zip
100% 6.17G/6.17G [01:33<00:00, 66.0MB/s]
/usr/local/lib/python3.11/dist-packages/gdown/__main__.py:140: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From (original): https://drive.google.com/uc?id=1fKpXrNflzGcNmOmwHpRxQp3XQdH5W7wF
From (redirected): https://drive.google.com/uc?id=1fKpXrNflzGcNmOmwHpRxQp3XQdH5W7wF&confirm=t&uuid=4a0dbab9-cefb-414b-aace-b6643bc1e8f3
To: /content/info

In [ ]:
!unzip sug_model_moe_t5.zip
!unzip info_model_moe_t5.zip
!unzip exp_model_moe_t5.zip
!unzip cause_model_moe_t5.zip

Archive:  sug_model_moe_t5.zip
   creating: content/flan_t5_suggestion_lora/
   creating: content/flan_t5_suggestion_lora/checkpoint-1023/
  inflating: content/flan_t5_suggestion_lora/checkpoint-1023/trainer_state.json  
  inflating: content/flan_t5_suggestion_lora/checkpoint-1023/config.json  
  inflating: content/flan_t5_suggestion_lora/checkpoint-1023/generation_config.json  
  inflating: content/flan_t5_suggestion_lora/checkpoint-1023/scheduler.pt  
  inflating: content/flan_t5_suggestion_lora/checkpoint-1023/special_tokens_map.json  
  inflating: content/flan_t5_suggestion_lora/checkpoint-1023/tokenizer_config.json  
  inflating: content/flan_t5_suggestion_lora/checkpoint-1023/optimizer.pt  
  inflating: content/flan_t5_suggestion_lora/checkpoint-1023/rng_state.pth  
  inflating: content/flan_t5_suggestion_lora/checkpoint-1023/training_args.bin  
  inflating: content/flan_t5_suggestion_lora/checkpoint-1023/spiece.model  
  inflating: content/flan_t5_suggestion_lora/checkpoint-1023

In [ ]:
# from google.colab import drive
# import os
# import shutil

# # Mount Google Drive
# drive.mount('/content/drive')


### Load DF

In [ ]:
train_file_path = '/content/train.json'
val_file_path = '/content/valid.json'
test_file_path = '/content/test.json'

with open(train_file_path, 'r') as f:
    train_data = json.load(f)
train_df = pd.DataFrame(train_data)
print("Train DataFrame loaded. Shape:", train_df.shape)

with open(val_file_path, 'r') as f:
    val_data = json.load(f)
val_df = pd.DataFrame(val_data)
print("Validation DataFrame loaded. Shape:", val_df.shape)


with open(test_file_path, 'r') as f:
    test_data = json.load(f)
test_df = pd.DataFrame(test_data)
print("Test DataFrame loaded. Shape:", test_df.shape)


Train DataFrame loaded. Shape: (2236, 7)
Validation DataFrame loaded. Shape: (959, 7)
Test DataFrame loaded. Shape: (640, 7)


### Understand

In [ ]:
# import json
# import pandas as pd

# # Define file paths for the train, validation, and test JSON files.
# train_file_path = '/content/train.json'
# val_file_path = '/content/valid.json'
# test_file_path = '/content/test.json'

# # Load train data
# with open(train_file_path, 'r') as f:
#     train_data = json.load(f)
# train_df = pd.DataFrame(train_data)
# print("Train DataFrame loaded. Shape:", train_df.shape)
# print("\nTrain DataFrame Columns:")
# print(train_df.columns.tolist())
# print("\nFirst 5 rows of Train DataFrame:")
# print(train_df.head())

# # Load validation data
# with open(val_file_path, 'r') as f:
#     val_data = json.load(f)
# val_df = pd.DataFrame(val_data)
# print("\nValidation DataFrame loaded. Shape:", val_df.shape)
# print("\nValidation DataFrame Columns:")
# print(val_df.columns.tolist())
# print("\nFirst 5 rows of Validation DataFrame:")
# print(val_df.head())

# # Load test data
# with open(test_file_path, 'r') as f:
#     test_data = json.load(f)
# test_df = pd.DataFrame(test_data)
# print("\nTest DataFrame loaded. Shape:", test_df.shape)
# print("\nTest DataFrame Columns:")
# print(test_df.columns.tolist())
# print("\nFirst 5 rows of Test DataFrame:")
# print(test_df.head())

# # A helper function to inspect dictionary-type columns
# def inspect_dict_column(df, col_name):
#     if col_name in df.columns:
#         # Filter rows where the column is not empty and is a dictionary
#         dict_items = df[col_name].dropna().apply(lambda x: x if isinstance(x, dict) else None).dropna()
#         if not dict_items.empty:
#             print(f"\nSample keys in '{col_name}' column:")
#             print(dict_items.head().apply(lambda x: list(x.keys())))
#         else:
#             print(f"\nNo dictionary content found in the column '{col_name}'.")
#     else:
#         print(f"\nColumn '{col_name}' does not exist in the DataFrame.")

# # Inspect dictionary-type columns in the train DataFrame
# inspect_dict_column(train_df, "labelled_summaries")
# inspect_dict_column(train_df, "labelled_answer_spans")

# # Repeat inspection for validation and test DataFrames if needed
# inspect_dict_column(val_df, "labelled_summaries")
# inspect_dict_column(val_df, "labelled_answer_spans")

# inspect_dict_column(test_df, "labelled_summaries")
# inspect_dict_column(test_df, "labelled_answer_spans")

# # Additional summary: info() and value counts for columns of interest
# print("\nTrain DataFrame info:")
# print(train_df.info())

# # Example: distribution of a key column (if available, e.g., 'question')
# if "question" in train_df.columns:
#     print("\nValue counts for 'question' column (first 10):")
#     print(train_df["question"].value_counts().head(10))


In [ ]:
import pandas as pd
from transformers import AutoTokenizer

# Load the tokenizer for Flan-T5 base
model_name = "google/flan-t5-base"
tokenizer = AutoTokenizer.from_pretrained(model_name)

# Define the perspective keys we want to analyze
perspectives = ["INFORMATION_SUMMARY", "SUGGESTION_SUMMARY", "CAUSE_SUMMARY", "EXPERIENCE_SUMMARY"]

# Initialize dictionaries to hold word counts and token counts per perspective
persp_word_counts = {p: [] for p in perspectives}
persp_token_counts = {p: [] for p in perspectives}

# Iterate over each row in the train DataFrame
for idx, row in train_df.iterrows():
    summaries = row.get("labelled_summaries", {})
    if isinstance(summaries, dict):
        for p in perspectives:
            if p in summaries:
                summary_text = summaries[p].strip()
                if summary_text:
                    # Word count: using simple .split() on whitespace
                    words = summary_text.split()
                    word_count = len(words)
                    # Tokenize to get token count
                    tokens = tokenizer.tokenize(summary_text)
                    token_count = len(tokens)

                    # Append counts
                    persp_word_counts[p].append(word_count)
                    persp_token_counts[p].append(token_count)

# Print the overall average lengths per perspective
for p in perspectives:
    if persp_word_counts[p]:
        avg_word_count = sum(persp_word_counts[p]) / len(persp_word_counts[p])
        avg_token_count = sum(persp_token_counts[p]) / len(persp_token_counts[p])
        print(f"{p}: Average word count: {avg_word_count:.2f}, Average token count: {avg_token_count:.2f}")
    else:
        print(f"{p}: No data found in training set.")


Leng of Generated Summaries

In [ ]:
import pandas as pd
from transformers import AutoTokenizer

# Specify the path to your CSV file
csv_path = "/content/3epoch.csv"  # e.g., "/content/generated_result.csv"

# Load the CSV file into a DataFrame
df = pd.read_csv(csv_path)

# Ensure that the relevant columns are present
if "perspective" not in df.columns or "generated_summary" not in df.columns:
    raise ValueError("CSV must contain 'perspective' and 'generated_summary' columns.")

# Load the Flan-T5-base tokenizer (or change to any tokenizer you prefer)
tokenizer = AutoTokenizer.from_pretrained("google/flan-t5-base")

# Define functions to compute word count and token count
def get_word_count(text):
    return len(str(text).split())

def get_token_count(text):
    return len(tokenizer.tokenize(str(text)))

# Compute word and token counts for each generated summary
df["word_count"] = df["generated_summary"].apply(get_word_count)
df["token_count"] = df["generated_summary"].apply(get_token_count)

# Group by perspective and calculate the average word and token counts
avg_lengths = df.groupby("perspective").agg({"word_count": "mean", "token_count": "mean"}).reset_index()

# Print the results
print("Average summary lengths per perspective:")
print(avg_lengths)


### Preprocess

In [ ]:
def preprocess_data(df, perspective="INFORMATION_SUMMARY"):
    inputs, targets = [], []

    for idx, row in df.iterrows():
        summary_dict = row["labelled_summaries"]
        if perspective in summary_dict:
            if isinstance(row["answers"], list):
                answer_text = "\n".join(answer.strip() for answer in row["answers"])
            else:
                answer_text = row["answers"].strip()

            # answer + question + context
            input_text = f"answer: {answer_text} question: {row['question']} context: {row['context']}"
            target_text = summary_dict[perspective]
            inputs.append(input_text.strip())
            targets.append(target_text.strip())

    return pd.DataFrame({"input_text": inputs, "target_text": targets})


### Model and *LoRa*

LoRA

In [ ]:
model_name = "google/flan-t5-base"
tokenizer = AutoTokenizer.from_pretrained(model_name)
base_model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

lora_config = LoraConfig(
    r=16,
    lora_alpha=32,
    target_modules=["q", "v"],
    lora_dropout=0.05,
    bias="none",
    task_type=TaskType.SEQ_2_SEQ_LM
)

model = get_peft_model(base_model, lora_config)

model.print_trainable_parameters()


Standard Model

In [ ]:
model_name = "google/flan-t5-base"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

total_params = sum(p.numel() for p in model.parameters())
trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/2.54k [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

...

config.json:   0%|          | 0.00/1.40k [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/990M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

Total parameters: 247577856
Trainable parameters: 247577856


### Train

In [ ]:
info_train_df = preprocess_data(train_df, "INFORMATION_SUMMARY")
info_val_df = preprocess_data(val_df, "INFORMATION_SUMMARY")

sugg_train_df = preprocess_data(train_df, "SUGGESTION_SUMMARY")
sugg_val_df = preprocess_data(val_df, "SUGGESTION_SUMMARY")

cause_train_df = preprocess_data(train_df, "CAUSE_SUMMARY")
cause_val_df = preprocess_data(val_df, "CAUSE_SUMMARY")

exp_train_df = preprocess_data(train_df, "EXPERIENCE_SUMMARY")
exp_val_df = preprocess_data(val_df, "EXPERIENCE_SUMMARY")

In [ ]:
def train_lora_model(train_df, val_df, perspective_name):
    train_dataset = Dataset.from_pandas(train_df)
    val_dataset = Dataset.from_pandas(val_df)

    def tokenize_function(example):
        model_inputs = tokenizer(
            example["input_text"],
            max_length=512,
            padding="max_length",
            truncation=True
        )
        with tokenizer.as_target_tokenizer():
            labels = tokenizer(
                example["target_text"],
                max_length=200,
                padding="max_length",
                truncation=True
            )
        model_inputs["labels"] = labels["input_ids"]
        return model_inputs

    tokenized_train = train_dataset.map(tokenize_function, batched=True)
    tokenized_val = val_dataset.map(tokenize_function, batched=True)

    output_dir = f"./flan_t5_{perspective_name.lower()}_lora"

# For LoRA
    # training_args = Seq2SeqTrainingArguments(
    #     output_dir=output_dir,
    #     num_train_epochs=3,
    #     per_device_train_batch_size=8,
    #     per_device_eval_batch_size=8,
    #     learning_rate=3e-4,
    #     evaluation_strategy="epoch",
    #     save_strategy="epoch",
    #     logging_dir=os.path.join(output_dir, "logs"),
    #     save_total_limit=2,
    #     predict_with_generate=True,
    #     report_to="none",
    #     logging_steps=20,
    #     generation_max_length=128,
    #     generation_num_beams=4
    # )

# For Standard

    training_args = Seq2SeqTrainingArguments(
        output_dir=output_dir,
        num_train_epochs=3,
        per_device_train_batch_size=4,
        per_device_eval_batch_size=4,
        learning_rate=3e-4,
        eval_strategy="epoch",
        save_strategy="epoch",
        logging_dir=os.path.join(output_dir, "logs"),
        save_total_limit=2,
        predict_with_generate=True,
        report_to="none",
        logging_steps=20,
        generation_max_length=128,
        generation_num_beams=4
    )


    data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

    trainer = Seq2SeqTrainer(
        model=model,
        args=training_args,
        train_dataset=tokenized_train,
        eval_dataset=tokenized_val,
        tokenizer=tokenizer,
        data_collator=data_collator
    )
    trainer.train()
    model.save_pretrained(output_dir)
    tokenizer.save_pretrained(output_dir)

    print(f"Finished training and saved model for: {perspective_name}")


Train Experts

In [ ]:
train_lora_model(info_train_df, info_val_df, "INFORMATION")

In [ ]:
train_lora_model(cause_train_df, cause_val_df, "CAUSE")

In [ ]:
train_lora_model(sugg_train_df, sugg_val_df, "SUGGESTION")

In [ ]:
train_lora_model(exp_train_df, exp_val_df, "EXPERIENCE")

Save Experts on Drive

In [ ]:
# drive_folder = "/content/drive/MyDrive/flan_t5_models_lora"

# os.makedirs(drive_folder, exist_ok=True)
# !zip -r sug_model_lora.zip /content/flan_t5_suggestion_lora
# !zip -r exp_model_lora.zip /content/flan_t5_experience_lora
# !zip -r cause_model_lora.zip /content/flan_t5_cause_lora
# !zip -r info_model_lora.zip /content/flan_t5_information_lora

# shutil.move("sug_model_lora.zip", os.path.join(drive_folder, "sug_model_lora.zip"))
# shutil.move("exp_model_lora.zip", os.path.join(drive_folder, "exp_model_lora.zip"))
# shutil.move("cause_model_lora.zip", os.path.join(drive_folder, "cause_model_lora.zip"))
# shutil.move("info_model_lora.zip", os.path.join(drive_folder, "info_model_lora.zip"))

# print("Model zips moved to Google Drive.")


### Test

In [ ]:
def expand_test_df(test_df):
    rows = []

    for _, row in test_df.iterrows():
        if isinstance(row["labelled_summaries"], dict):
            for perspective, summary in row["labelled_summaries"].items():
                if isinstance(summary, str) and summary.strip():
                    rows.append({
                        "uri": row["uri"],
                        "question": row["question"],
                        "context": row["context"],
                        "perspective": perspective,
                        "summary": summary.strip()
                    })

    expanded_df = pd.DataFrame(rows)
    return expanded_df

expanded_test_df = expand_test_df(test_df)

print(len(test_df),len(expanded_test_df))

640 1252
       uri                                           question  \
0  1309809                                    what is orgasm?   
1  1846165  when mutiple women live in the same house for ...   
2  1564117  I am extremely tired and an slowly gaining wei...   
3  1564117  I am extremely tired and an slowly gaining wei...   
4  1564117  I am extremely tired and an slowly gaining wei...   

                                             context          perspective  \
0                                                     INFORMATION_SUMMARY   
1  when mutiple women live in the same house for ...  INFORMATION_SUMMARY   
2              I am 27, very active!Recently married  INFORMATION_SUMMARY   
3              I am 27, very active!Recently married        CAUSE_SUMMARY   
4              I am 27, very active!Recently married   SUGGESTION_SUMMARY   

                                             summary  
0  For information purposes, an orgasm is a pleas...  
1  For information purposes

In [ ]:
def generate_summary(row, index):
    summary_type = row["perspective"]
    print(f"Processing [{index + 1}] - Perspective: {summary_type}")

    if summary_type not in loaded_models:
        return None

    model, tokenizer = loaded_models[summary_type]

    prompt_parts = []
    if pd.notna(row["question"]):
        prompt_parts.append(f"Question: {row['question']}")
    if pd.notna(row["context"]):
        prompt_parts.append(f"Context: {row['context']}")
    if pd.notna(row["context"]):
        prompt_parts.append(f"Answers: {' '.join(row['context'])}")

    input_text = " ".join(prompt_parts)
    inputs = tokenizer(input_text, return_tensors="pt", truncation=True, padding=True).to(model.device)

    with torch.no_grad():
        # outputs = model.generate(**inputs, max_new_tokens=100)

        # For Longer
        outputs = model.generate(
            **inputs,
            min_length=70,
            max_new_tokens=150,
            num_beams=4,
            early_stopping=True
        )

        return tokenizer.decode(outputs[0], skip_special_tokens=True)



In [ ]:
from google.colab import files

In [ ]:

generated = []
file_countx = 0
for i, row in expanded_test_df.iterrows():
    summary = generate_summary(row, i)
    generated.append(summary)

    expanded_test_df.at[i, 'generated_summary'] = summary
    expanded_test_df.to_csv("moe_test_predictions.csv", index=False)
    file_countx = (file_countx + 1) % 200
    if file_countx==1 : files.download('moe_test_predictions.csv')
    print(f"Iteration {i+1} complete, CSV updated.")

# files.download('moe_test_predictions.csv')
# # Save results
expanded_test_df["generated_summary"] = generated
expanded_test_df.to_csv("moe_test_predictions.csv", index=False)
print("\nAll summaries generated and saved to csv")


Processing [1] - Perspective: INFORMATION_SUMMARY


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Iteration 1 complete, CSV updated.
Processing [2] - Perspective: INFORMATION_SUMMARY
Iteration 2 complete, CSV updated.
Processing [3] - Perspective: INFORMATION_SUMMARY
Iteration 3 complete, CSV updated.
Processing [4] - Perspective: CAUSE_SUMMARY
Iteration 4 complete, CSV updated.
Processing [5] - Perspective: SUGGESTION_SUMMARY
Iteration 5 complete, CSV updated.
Processing [6] - Perspective: INFORMATION_SUMMARY
Iteration 6 complete, CSV updated.
Processing [7] - Perspective: SUGGESTION_SUMMARY
Iteration 7 complete, CSV updated.
Processing [8] - Perspective: INFORMATION_SUMMARY
Iteration 8 complete, CSV updated.
Processing [9] - Perspective: INFORMATION_SUMMARY
Iteration 9 complete, CSV updated.
Processing [10] - Perspective: SUGGESTION_SUMMARY
Iteration 10 complete, CSV updated.
Processing [11] - Perspective: INFORMATION_SUMMARY
Iteration 11 complete, CSV updated.
Processing [12] - Perspective: CAUSE_SUMMARY
Iteration 12 complete, CSV updated.
Processing [13] - Perspective: SUGGESTI

<IPython.core.display.Javascript object>

...

<IPython.core.display.Javascript object>

Iteration 1201 complete, CSV updated.
Processing [1202] - Perspective: INFORMATION_SUMMARY
Iteration 1202 complete, CSV updated.
Processing [1203] - Perspective: INFORMATION_SUMMARY
Iteration 1203 complete, CSV updated.
Processing [1204] - Perspective: INFORMATION_SUMMARY
Iteration 1204 complete, CSV updated.
Processing [1205] - Perspective: SUGGESTION_SUMMARY
Iteration 1205 complete, CSV updated.
Processing [1206] - Perspective: INFORMATION_SUMMARY
Iteration 1206 complete, CSV updated.
Processing [1207] - Perspective: INFORMATION_SUMMARY
Iteration 1207 complete, CSV updated.
Processing [1208] - Perspective: SUGGESTION_SUMMARY
Iteration 1208 complete, CSV updated.
Processing [1209] - Perspective: INFORMATION_SUMMARY
Iteration 1209 complete, CSV updated.
Processing [1210] - Perspective: INFORMATION_SUMMARY
Iteration 1210 complete, CSV updated.
Processing [1211] - Perspective: CAUSE_SUMMARY
Iteration 1211 complete, CSV updated.
Processing [1212] - Perspective: SUGGESTION_SUMMARY
Iteratio

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>


✅ All summaries generated and saved to moe_test_predictions.csv


### Eval

In [ ]:
df = pd.read_csv("moe_test_predictions.csv")

df = df.dropna(subset=["generated_summary", "summary"])
df["generated_summary"] = df["generated_summary"].astype(str).str.strip()
df["summary"] = df["summary"].astype(str).str.strip()

print("\nCalculating OVERALL scores...")

P, R, F1 = bert_score(
    cands=df["generated_summary"].tolist(),
    refs=df["summary"].tolist(),
    lang="en",
    verbose=True,
    rescale_with_baseline=True,
)

overall_bleu = corpus_bleu(df["generated_summary"].tolist(), [df["summary"].tolist()]).score

print(f"\nOVERALL RESULTS:")
print(f"BERTScore - Precision: {P.mean():.4f}, Recall: {R.mean():.4f}, F1: {F1.mean():.4f}")
print(f"BLEU-4    - Score: {overall_bleu:.2f}")


perspective_scores = defaultdict(dict)

for perspective in df["perspective"].unique():
    sub_df = df[df["perspective"] == perspective]

    if sub_df.empty:
        continue

    P_sub, R_sub, F1_sub = bert_score(
        cands=sub_df["generated_summary"].tolist(),
        refs=sub_df["summary"].tolist(),
        lang="en",
        verbose=False,
        rescale_with_baseline=True,
    )

    bleu_sub = corpus_bleu(sub_df["generated_summary"].tolist(), [sub_df["summary"].tolist()]).score

    perspective_scores[perspective]["bert_precision"] = P_sub.mean().item()
    perspective_scores[perspective]["bert_recall"] = R_sub.mean().item()
    perspective_scores[perspective]["bert_f1"] = F1_sub.mean().item()
    perspective_scores[perspective]["bleu_4"] = bleu_sub

    print(f"{perspective}:")
    print(f"  BERTScore - P: {P_sub.mean():.4f}, R: {R_sub.mean():.4f}, F1: {F1_sub.mean():.4f}")
    print(f"  BLEU-4    - {bleu_sub:.2f}")
    print()

print("\nEvaluation Complete")


<ipython-input-1-74ba693c7680>:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["generated_summary"] = df["generated_summary"].astype(str).str.strip()
<ipython-input-1-74ba693c7680>:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["summary"] = df["summary"].astype(str).str.strip()



Calculating OVERALL scores...


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/482 [00:00<?, ?B/s]

...

CAUSE_SUMMARY:
  BERTScore - P: 0.0247, R: 0.2040, F1: 0.1109
  BLEU-4    - 2.52



Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


SUGGESTION_SUMMARY:
  BERTScore - P: 0.0409, R: 0.1396, F1: 0.0869
  BLEU-4    - 3.18



Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


EXPERIENCE_SUMMARY:
  BERTScore - P: 0.0027, R: 0.1891, F1: 0.0915
  BLEU-4    - 6.01


Evaluation Complete


### One Line

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model_paths = {
    "INFORMATION_SUMMARY": "./flan_t5_information_lora",
    "CAUSE_SUMMARY": "./flan_t5_cause_lora",
    "SUGGESTION_SUMMARY": "./flan_t5_suggestion_lora",
    "EXPERIENCE_SUMMARY": "./flan_t5_experience_lora",
}

loaded_models = {}
loaded_tokenizers = {}

def load_model_for_perspective(perspective):
    if perspective not in model_paths:
        raise ValueError(f"Perspective '{perspective}' is not supported.")

    if perspective not in loaded_models:
        tokenizer = AutoTokenizer.from_pretrained(model_paths[perspective])
        model = AutoModelForSeq2SeqLM.from_pretrained(model_paths[perspective]).to(device)
        model.eval()
        loaded_models[perspective] = model
        loaded_tokenizers[perspective] = tokenizer

    return loaded_tokenizers[perspective], loaded_models[perspective]

def generate_summary(question, answer, perspective):
    tokenizer, model = load_model_for_perspective(perspective)

    input_text = f"question: {question}  context: {answer}"
    inputs = tokenizer(input_text, return_tensors="pt", truncation=True, padding=True).to(device)

    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            min_length=70,
            max_new_tokens=150,
            num_beams=4,
            early_stopping=True
        )

    return tokenizer.decode(outputs[0], skip_special_tokens=True)


In [ ]:
question = "What are the symptoms of high blood pressure?"
answer = "High blood pressure often has no symptoms. In some cases, people may experience headaches, shortness of breath, or nosebleeds."
perspective = "INFORMATION_SUMMARY"

summary = generate_summary(question, answer, perspective)
print(f"Generated Summary [{perspective}]:\n{summary}")
